# Node A: User Interface


This node implements an action client, allowing the user to set a target (x, y) or to cancel it. It uses the feedback/status of the action server to know when the target has been reached. The node also publishes the robot position and velocity as a custom message (x,y, vel_x, vel_z), by relying on the values published on the topic /odom.

Additionally, for the Resarch Track II assignment other funcionalities have been implemented. Buttons for robot position, set goals and cancelled goals information have been created. There is also a live plot for the robot position and the active target. Additionally, a plot for the number of set and cancelled goals.

### Import statements


In [ ]:
from __future__ import print_function
import rospy
import actionlib
import assignment_2_2023.msg
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
from actionlib_msgs.msg import GoalStatus
from assignment_2_2023.msg import PlanningFeedback, RobotState
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib widget

# Robot position
x_rob = 0
y_rob = 0
# Lists for all the set and cancelled goals
setgoals = []
cancelgoals = []

### Set the goal function

In [ ]:
def set_goal(client, x, y):
    goal = assignment_2_2023.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y
    client.send_goal(goal)
    
    setgoals.append((x, y))
    vis.targets_x.append(x)
    vis.targets_y.append(y)
    

### Cancel the goal function

In [ ]:
def cancel_goal(client, x, y):
    if client and client.get_state() == GoalStatus.ACTIVE:
        client.cancel_goal()
        client.wait_for_result()  # Wait for the cancellation to be processed
        
        cancelgoals.append((x, y))
        return True
    else:
        print("No active goal to cancel.")
        return False

### Callback function for the odom topic

In [ ]:
def odom_callback(odom_msg,robot_state_publisher):
    global x_rob, y_rob, client
    # Extract position and velocity of the robot from the odom message
    x = odom_msg.pose.pose.position.x
    y = odom_msg.pose.pose.position.y
    vel_x = odom_msg.twist.twist.linear.x
    vel_z = odom_msg.twist.twist.angular.z
    
    # Saving robot position to different global variables
    x_rob = odom_msg.pose.pose.position.x
    y_rob = odom_msg.pose.pose.position.y

    # Publish the position and velocity on the RobotState topic
    robot_state_msg = RobotState(x=x, y=y, vel_x=vel_x, vel_z=vel_z) # Write the message
    robot_state_publisher.publish(robot_state_msg) # Publish the message
    
    # Append new x and y values
    vis.y_data.append(odom_msg.pose.pose.position.y)
    vis.x_data.append(odom_msg.pose.pose.position.x)

    # Get the goal status and print the info 
    goal_status = client.get_state()

    # If the goal has been reached            
    if goal_status == GoalStatus.SUCCEEDED:
        textarea.value = "Goal is reached."

        # Remove the target from the list
        vis.targets_x.pop()
        vis.targets_y.pop()

    # If the goal has been canceled    
    elif goal_status == GoalStatus.PREEMPTED:
        textarea.value = "Goal is cancelled."

        # Remove the target from the list
        vis.targets_x.pop()
        vis.targets_y.pop()
    
    # If the goal is active  
    elif goal_status == GoalStatus.ACTIVE:
        textarea.value = "Goal is active."
    
    # If the goal is not active      
    elif goal_status == GoalStatus.LOST:
        textarea.value = "Goal is not active."
    
    # If the goal status is unknown 
    else:
        textarea.value = "Unknown goal state. Status code: {}".format(goal_status)

### Robots position

Let the user know the position of the robot by clicking on the button 'Current robot position'.

In [ ]:
# Create button widget
button = widgets.Button(description="Current robot position")

# Output widget to display output
output = widgets.Output()

# Function to handle button click event
def on_button_clicked(b):
    with output:
        # Clear previous output
        output.clear_output(wait=True)
        print(f"\nCurrent robot position: x = {x_rob:.4f}, y = {y_rob:.4f}")
        
# Attach event handler to the button widget
button.on_click(on_button_clicked)

# Display button widget and output
display(button, output)

### Set and cancelled goals

Let the user know all targets that have been set and cancelled in the environment by clicking on the button 'Set goals' and 'Cancelled goals' respectively.

In [ ]:
# Create button widget
button1 = widgets.Button(description="Set goals")
button2 = widgets.Button(description="Cancelled goals")

# Output widgets to display output
output1 = widgets.Output()
output2 = widgets.Output()

# Function to handle button 1 click event
def on_button1_clicked(b):
    with output1:
        # Clear previous output
        output1.clear_output(wait=True)
        
        print(f"\nSet goals")
        for x, y in setgoals:
            print(f"\nx = {x:.4f}, y = {y:.4f}")
            
# Function to handle button 2 click event
def on_button2_clicked(b):
    with output2:
        # Clear previous output
        output2.clear_output(wait=True)
        
        print(f"\nCancelled goals")
        for x, y in cancelgoals:
            print(f"\nx = {x:.4f}, y = {y:.4f}")
            print(len(cancelgoals))

# Attach event handler to the button widget
button1.on_click(on_button1_clicked)
button2.on_click(on_button2_clicked)

# Display button widgets and output widgets separately
display(widgets.VBox([button1, output1]))
display(widgets.VBox([button2, output2]))  
         

### Plots: Robot positions and set and cancelled targets

In [ ]:
# Class for both plots
class Visualiser:
    def __init__(self):
        self.fig, (self.ax1, self.ax2) = plt.subplots(1,2)
        self.ln, = self.ax1.plot([], [], 'ro')
        self.x_pts, = self.ax1.plot([], [], 'x')
        self.x_data, self.y_data = [], []
        self.targets_x, self.targets_y = [], []
        
    def plot_init(self):
        self.ax1.set_xlim(-10.0, 10.0)
        self.ax1.set_ylim(-10.0, 10.0)
        self.ax1.grid(True)
        
        self.ax2.set_ylim(0, 10)
        
        self.bar_labels = ['Set goals', 'Cancelled goals']
        self.bar_colors = ['tab:blue', 'tab:red']
        self.setgoals, self.cancelgoals = [], []
        self.bar = self.ax2.bar(self.bar_labels, [0,0], label=self.bar_labels, color=self.bar_colors)
        
        return self.ln, self.bar
    
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        self.x_pts.set_data(self.targets_x, self.targets_y)
        for bb, ii in zip(self.bar, [len(setgoals),len(cancelgoals)]):
            bb.set_height(ii)

        return self.ln, self.bar
    
vis = Visualiser()

# Create FuncAnimation with update_plot and plot_init methods
ani = FuncAnimation(vis.fig, vis.update_plot, init_func = vis.plot_init)
# Show the plot
plt.show()

## Main function

In [ ]:
# Initializes a rospy node so that the SimpleActionClient can publish and subscribe over ROS
rospy.init_node('user_interface')

# Create the SimpleActionClient
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
        
# Wait until the action server has started up and started listening for goals
client.wait_for_server()

# Publisher to the RobotState custom message
robot_state_publisher = rospy.Publisher('/RobotState', RobotState, queue_size=1)

# Create a Textarea widget to display the goal status
textarea = widgets.Text(description = 'Goal state', disabled = True,
                       layout = widgets.Layout(height = '30px', width = '270px'))

# Subscribe to the /odom topic
odom_subscriber = rospy.Subscriber('/odom', Odometry, odom_callback, robot_state_publisher, queue_size=1)

# Header widget
header_label = widgets.Label(value='Enter the goal', style={'font-weight': 'bold'})

# Create sliders and outputs
x_slider = widgets.IntSlider(
    min=-10,
    max=10,
    description='x coordinate',
    continuous_update=False
)

y_slider = widgets.IntSlider(
    min=-10,
    max=10,
    description='y coordinate',
    continuous_update=False
)

output_slider = widgets.Output()

cancel_dropdown = widgets.Dropdown(
    options=['(None)', 'No', 'Yes'],
    value='(None)',
    disabled=False,
    layout={'width': 'auto', 'description_width': 'initial'}
)
    
# Create a VBox to hold x_slider, output1, and the cancel_dropdown
x_box = widgets.VBox([header_label, x_slider, output_slider])
display(x_box)

# Create a VBox to hold y_slider and output2
y_box = widgets.VBox([y_slider, output_slider, widgets.HBox([widgets.Label('Do you want to cancel the goal?'), cancel_dropdown])])
display(y_box)
    
def on_value_change(change):
    if x_slider.value is not None and y_slider.value is not None:
        with output_slider:
            # Send the goal
            set_goal(client, x_slider.value, y_slider.value)

# Observe only the y coordinate value
#x_slider.observe(on_value_change, names='value')
y_slider.observe(on_value_change, names='value')

def on_cancel_change(change):
    if change['new'] == 'Yes':
        success = cancel_goal(client, x_slider.value, y_slider.value)

# Attach the change event handler
cancel_dropdown.observe(on_cancel_change, names='value')

# Display the textarea widget
display(textarea)